In [ ]:
# Reference: https://github.com/bayesian-optimization/BayesianOptimization/blob/master/examples/advanced-tour.ipynb
# $ conda install -c conda-forge bayesian-optimization

import json
import subprocess

from bayes_opt.event import Events
from bayes_opt import BayesianOptimization, UtilityFunction
from bayes_opt.logger import JSONLogger

RANDOM_STATE = 1
VERBOSE = 2
FILE_PATH_LOG = "./bayesian_optimization/logs.log.json"
FILE_PATH_PARAMETER = "./bayesian_optimization/parameters.json"
N_LOOP = 20

class UtilityFuncFactory():
    def __init__(self, kind_list=["ucb", "ei", "poi"]):
        self.kind_list = kind_list
        self.kind = None

    def construct(self, i: int) -> UtilityFunction:
        KAPPA = 2.576
        # Use different acquisition function in cyclic
        self.kind = self.kind_list[i % len(self.kind_list)]
        utility = UtilityFunction(kind=self.kind, kappa=KAPPA)
        return utility

class Experiment():
    def __init__(self, is_subprocess=False):
        self.is_subprocess = is_subprocess
        self.bounds = {"x1": (-2, 2), "x2": (-3, 3)}

    def exec(self, next_point) -> float:
        if self.is_subprocess:
            with open(FILE_PATH_PARAMETER, "w") as f:
                json.dump(next_point, f)
            proc = subprocess.run(['python', 'experiment.py'], capture_output=True)
            return float(proc.stdout.decode('utf-8'))
        else:
            x1 = next_point["x1"]
            x2 = next_point["x2"]
            return - (x1 ** 2) - (x2 -1) ** 2 + 1


# Construct instance
utility_func_factory = UtilityFuncFactory()
experiment = Experiment()
bounds = experiment.bounds

optimizer = BayesianOptimization(
    f=None,
    pbounds=bounds,
    random_state=RANDOM_STATE,
    allow_duplicate_points=True,
    verbose=VERBOSE
)

# Set logger
logger = JSONLogger(path=FILE_PATH_LOG, reset=True)
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

for i_loop in range(N_LOOP):
    # Suggest next point
    utility = utility_func_factory.construct(i_loop)
    params = optimizer.suggest(utility)
    print(f"loop: {i_loop}, acq func: {utility.kind}")
    # Experiment
    target = experiment.exec(params)
    # Register the result
    optimizer.register(params=params, target=target)
    print(f"params: {params}, target: {target}")
print("The best:", optimizer.max)